In [1]:
import time
import numpy as np 
from zmqRemoteApi import RemoteAPIClient

In [2]:
def degree_To_radian(degree):
    return (degree * np.pi/180)


def FK(theta, alfa, a, d):
    
    Cos_theta = np.cos(theta)
    Sin_theta = np.sin(theta)
    
    Cos_alfa = np.cos(alfa)
    Sin_alfa = np.sin(alfa)
    
    Transformation_mat = np.array([
        
                                    [Cos_theta,-Sin_theta*Cos_alfa,Sin_theta*Sin_alfa,a*Cos_theta],
                                    [Sin_theta,Cos_theta*Cos_alfa,-Cos_theta*Sin_alfa,a*Sin_theta],
                                    [0,Sin_alfa,Cos_alfa,d],
                                    [0,0,0,1]
    ])
    
    return np.round(Transformation_mat.reshape(4,4),2)

In [3]:
def Calculate_Jacopian_P(Trans_mat_arr):
    
    O_0 = np.zeros([3,1])
    Jp_size = len(Trans_mat_arr)
    Z_0 = np.array([[0,0,1]]).reshape(3,1)
    
    Z_arr = [z[0:3,2] for z in Trans_mat_arr]
    Z_arr.insert(0,Z_0)
    
    O_arr = [o[0:3,3] for o in Trans_mat_arr]
    O_arr.insert(0,O_0)
    
    O_last = O_arr[-1].reshape(3,1)
    
    jp_mat = np.array([
        [np.cross(Z_arr[i].reshape(3,1) , (O_last-O_arr[i].reshape(3,1)),axis=0) for i in range(Jp_size)]
    ])
        
    return jp_mat.reshape(3,Jp_size).transpose()
    
def Calculate_Jacopian_r(Trans_mat_arr):
    
    Z_0 = np.array([[0,0,1]])
    Jp_size = len(Trans_mat_arr)
        
    Z_arr = [Trans_mat_arr[z][0:3,2] for z in range(Jp_size-1)]
    Z_arr.insert(0,Z_0)
    
    Jr_mat = np.array([[z.reshape(3,1) for z in Z_arr]])
    
    return Jr_mat.reshape(3,Jp_size).transpose()

def Calculate_J(Trans_mat_arr):
    J_size = len(Trans_mat_arr)
    J = np.array([
                    [Calculate_Jacopian_P(Trans_mat_arr),
                     Calculate_Jacopian_r(Trans_mat_arr)]
                ])
    
    return J.reshape(6,J_size)
    

In [4]:
def Calculate_0n_Matrix(Trans_mat_arr):
    
    n = len(Trans_mat_arr)
    T_0 = np.identity(4)
    
    result_mat = []
    
    for i in range(n):
        T_0 = np.matmul(T_0,Trans_mat_arr[i])
        result_mat.append( T_0 )
        
    return result_mat
    

In [5]:
def Calculate_Delta_X(Td, T0):
    
    Pd = Td[0:3,3]
    P0 = T0[0:3,3]
    
    Rd = Td[0:3,0:3]
    R0 = T0[0:3,0:3]
    
    I = np.identity(3)
    
    Pos_list = np.array([[d-c for d,c in zip(Pd,P0)]])
    Pos_list = Pos_list.reshape(3,1)
    
    DR = np.matmul(Rd,R0.transpose())
    skew = np.subtract(DR , I)
    skew = np.round(skew,2)
    print("skew = \n",skew)
    
    vec = np.array([[abs(skew[1,2]),abs(skew[0,2]) ,abs(skew[0,1])]])
    #vec = np.array([1,1,1])
    #vec = np.vectorize(skew)
    
    print("vec = \n",vec)
    vec = vec.reshape(3,1)
    
    Delta_X = np.array([Pos_list,vec])
        
    return np.round(Delta_X.reshape(6,1),2)  

In [6]:
def Calculate_IK(dX,J,q0):
    # qd = q0 + J+ * d_X
    j_pinv = np.linalg.pinv(J)
    
    q0_arr = np.array([q0])
    q0_arr = q0_arr.reshape(6,1)
    
    qd = np.add(q0_arr,np.matmul(np.round(j_pinv,2),dX)) # 6*1
    
    #qd = [the[0] for the in temp]
    
    return np.round(qd.reshape(1,6),2)

In [16]:
client = RemoteAPIClient()
sim = client.getObject('sim')

# When simulation is not running, ZMQ message handling could be a bit
# slow, since the idle loop runs at 8 Hz by default. So let's make
# sure that the idle loop runs at full speed for this program:
defaultIdleFps = sim.getInt32Param(sim.intparam_idle_fps)
sim.setInt32Param(sim.intparam_idle_fps, 0)

1

In [17]:
# set Object name 
joint_names = [f"joint_{i+1}"  for i in range(6)]

joint_handels = [sim.getObjectHandle(joint) for joint in joint_names]
# set Object handel 
EE_handle = sim.getObjectHandle("EE_h")
EE_h_Pos = sim.getObjectPosition(EE_handle,joint_handels[0])


joint_Position = [sim.getJointPosition(handle) for handle in joint_handels]

joint_6_pos = sim.getObjectPosition(joint_handels[-1],joint_handels[0] )

joint_6_pos = np.round(joint_6_pos,2)

# T0
EE_Trans = np.array([ sim.getObjectMatrix( EE_handle ,joint_handels[0]) ])
EE_Trans = np.round(EE_Trans.reshape(3,4),2)

Target_handle = sim.getObjectHandle("Cuboid")

# print("joint_Position = \n",np.round(joint_Position,2))
# print("EE_Trans = \n",np.round(EE_Trans.reshape(3,4),2))
# print("EE_h_Pos = \n",np.round(EE_h_Pos,2))

for handle in joint_handels:
    sim.setObjectInt32Parameter(handle ,2001,~0) #to enable the position control

In [18]:
# init DH-parameters
q0 = [0,0,0,0,0,0]
alpha = [90,0,0,90,-90,0]
D = [0.1519,0,0,0.11235,0.08535,0.0819]
A = [0,-0.24365,-0.21325,0,0,0]


In [19]:
# calc Transformation matrix using FK
Trans_list = [FK(degree_To_radian(theta),degree_To_radian(alp),a,d) for theta,alp,a,d in zip(q0,alpha,A,D)]

T_0n = Calculate_0n_Matrix(Trans_list)

# T_0n = [ np.array([ sim.getObjectMatrix(joint_handels[i],joint_handels[0] )]).reshape(3,4) for i in range(1,6)]

# T_0n.insert(0,np.array([ sim.getObjectMatrix( joint_handels[0] , -1 ) ]).reshape(3,4) )

# for i in range(6):
#      T_0n[i] = np.round(T_0n[i],2)

# print("qd = \n",qd,"\n\n")
print("T_0n[5] = \n",T_0n[-1],"\n\n")
print("EE_Trans = \n",EE_Trans,"\n\n")
print("EE_h_Pos = \n",np.round(EE_h_Pos,2),"\n\n")
print("joint_6_pos = \n",joint_6_pos)
print("t00 = \n",T_0n[0])
# print("delta_X = \n",delta_X)

# for t in Trans_list:
#     print("t = \n",t,"\n")

# calc T disierd
# T_d = FK(degree_To_radian(30),degree_To_radian(45),0,0)
T_d = np.array([ sim.getObjectMatrix( Target_handle ,joint_handels[0])])
T_d = T_d.reshape(3,4)
T0_6 = EE_Trans

T_0n[5] = 
 [[ 1.    0.    0.   -0.45]
 [ 0.    0.   -1.   -0.19]
 [ 0.    1.    0.    0.06]
 [ 0.    0.    0.    1.  ]] 


EE_Trans = 
 [[ 0.    1.    0.   -0.  ]
 [ 0.   -0.    1.    0.19]
 [ 1.   -0.    0.    0.55]] 


EE_h_Pos = 
 [-0.    0.19  0.55] 


joint_6_pos = 
 [-0.    0.11  0.55]
t00 = 
 [[ 1.    0.    0.    0.  ]
 [ 0.    0.   -1.    0.  ]
 [ 0.    1.    0.    0.15]
 [ 0.    0.    0.    1.  ]]


In [20]:
# claculate Jacopian matrix
J = Calculate_J(T_0n)

# calculate delta X matrix  
delta_X = Calculate_Delta_X(T_d,EE_Trans)

print("delta_X = \n ",delta_X)

# calculate qd using IK 
qd = Calculate_IK(delta_X,J,q0)

print(qd)

print("np.linalg.norm(delta_X) =",np.round(np.linalg.norm(delta_X),2))

# print("np.linalg.norm(delta_X) = \n",np.round(np.linalg.norm(delta_X),2))

skew = 
 [[ 0.  0. -0.]
 [-0. -1. -1.]
 [ 0.  1. -1.]]
vec = 
 [[1. 0. 0.]]
delta_X = 
  [[ 0.2 ]
 [ 0.11]
 [-0.45]
 [ 1.  ]
 [ 0.  ]
 [ 0.  ]]
[[ 0.32 -0.57  0.26  0.28 -0.99  0.  ]]
np.linalg.norm(delta_X) = 1.12


In [ ]:
while ~( np.isclose( np.linalg.norm(delta_X),0.1 ,rtol=0.01)  ):
    
    #q0 = qd.copy()


#     Trans_list.clear()
    
#     Trans_list = [FK(theta,degree_To_radian(alp),a,d) for theta,alp,d,a in zip(q0[0],alpha,D,A)]
    
#     T_0n = Calculate_0n_Matrix(Trans_list)

    T_0n.clear()

    T_0n = [ np.array([ sim.getObjectMatrix(joint_handels[i],joint_handels[0] )]).reshape(3,4) for i in range(1,6)]

    T_0n.insert(0,np.array([ sim.getObjectMatrix( joint_handels[0] , -1 ) ]).reshape(3,4) )

    for i in range(6):
         T_0n[i] = np.round(T_0n[i],2)

    J = Calculate_J(T_0n)
    delta_X = Calculate_Delta_X(T_d,T_0n[-1])
    
    print("delta_X = \n",delta_X)
    print("np.linalg.norm(delta_X) =",np.round(np.linalg.norm(delta_X),2))
    
    qd = Calculate_IK(delta_X,J,q0)
    print("qd = \n",qd,"\n\n")    

In [21]:
# gripper 

#yum_gripper_handle = sim.getObjectHandle("openCloseJoint")

connector=sim.getObject('./attachPoint')
objectSensor=sim.getObject('./attachProxSensor')

#Target_Object_handle = 


index=0
while True: 
    
    shape=sim.getObjects(index,sim.object_shape_type)
    
    if shape==-1:
        break
        
    cond_1 = sim.getObjectInt32Param(shape,sim.shapeintparam_static)
    cond_2 = sim.getObjectInt32Param(shape,sim.shapeintparam_respondable)
    cond_3 = sim.checkProximitySensor(objectSensor,shape)
    
    if ((cond_1==0) and (cond_2!=0) and (cond_3==1)):
         # Ok, we found a non-static respondable shape that was detected
        attachedShape=shape
         # Do the connection:
        sim.setObjectParent(attachedShape,connector,true)
        break
    index +=1


# c) And just before opening the gripper again, detach the previously attached shape:
#sim.setObjectParent(attachedShape,-1,true)


In [22]:
print(q0)
print(qd)

[0, 0, 0, 0, 0, 0]
[[ 0.32 -0.57  0.26  0.28 -0.99  0.  ]]


In [26]:

client.setStepping(False)
sim.startSimulation()


for (handle,degree) in zip(joint_handels,qd[0]):
    sim.startSimulation()
#     # close
#     sim.setJointTargetVelocity(yum_gripper_handle,-0.02)
    sim.setJointTargetPosition(handle, degree ) # degree by radian
    

In [ ]:
#open
#sim.setJointTargetVelocity(yum_gripper_handle,0.02)

In [ ]:
sim.stopSimulation()